In [109]:
import pandas as pd 
import matplotlib.pyplot as plt
#plt.rcParams['font.family'] = 'AppleGothic' # 폰트 변경
#plt.rcParams['axes.unicode_minus'] = False # 축 값 마이너스 깨짐 해결

from torch.nn import Transformer
from torch import nn
import torch
import math
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import mean_absolute_percentage_error ,mean_squared_error

In [110]:
#path = '~/workplace/datas/energy/'
path = '~/Desktop/workplace/datas/energy/'

train_df = pd.read_csv(path + 'final_data.csv',index_col=0)
y_test = pd.read_csv(path + 'y_test.csv', index_col=0)

In [111]:
train_df.reset_index(inplace=True)

In [112]:
train_df = train_df.rename(columns={'index':'기준일시'})

In [113]:
train_df.tail()

,기준일시,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),일조(hr),일사(MJ/m2),적설(cm),전운량(10분위),지면온도(°C),현재수요(MW)
547516,2023-03-19 23:35:00,4.946842,0.0,0.862368,68.204386,1006.203070,0.08617,0.060784,0.0,2.612281,3.775088,58627.16
547517,2023-03-19 23:40:00,4.870526,0.0,0.864211,68.477193,1006.187719,0.08617,0.060784,0.0,2.645614,3.722807,57941.24
547518,2023-03-19 23:45:00,4.794211,0.0,0.866053,68.750000,1006.172368,0.08617,0.060784,0.0,2.678947,3.670526,57726.21
547519,2023-03-19 23:50:00,4.717895,0.0,0.867895,69.022807,1006.157018,0.08617,0.060784,0.0,2.712281,3.618246,57755.40
547520,2023-03-19 23:55:00,4.641579,0.0,0.869737,69.295614,1006.141667,0.08617,0.060784,0.0,2.745614,3.565965,57323.79


In [114]:
train_df['기준일시'] = pd.to_datetime(train_df['기준일시'])

In [115]:
train_df['hour'] = train_df['기준일시'].dt.hour
train_df['month'] = train_df['기준일시'].dt.month
train_df['year'] = train_df['기준일시'].dt.year
train_df['day'] = train_df['기준일시'].dt.day
train_df['min'] = train_df['기준일시'].dt.minute

In [ ]:
#더미를 같이하고 나눠줘야 컬럼수가 같구나....

In [117]:
train_df = pd.get_dummies(train_df, columns=['hour', 'month', 'year', 'day', 'min'])

In [118]:
train_df.shape

(547521, 97)

In [119]:
encoded_test_df = train_df[train_df['기준일시'] >= '2023-03-13']
encoded_train_df = train_df[train_df['기준일시'] < '2023-03-13']

In [120]:
encoded_test_df.drop(columns=['기준일시'], inplace=True)
encoded_train_df.drop(columns=['기준일시'], inplace=True)

/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_17942/542440738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encoded_test_df.drop(columns=['기준일시'], inplace=True)
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_17942/542440738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encoded_train_df.drop(columns=['기준일시'], inplace=True)


In [121]:
encoded_test_df.shape , encoded_train_df.shape

((2016, 96), (545505, 96))

In [122]:
encoded_train_df.to_csv(path + 'encoded_train_df.csv')
encoded_test_df.to_csv(path + 'encoded_test_df.csv')